In [ ]:
import cv2
import os
import numpy as np


# Ruta de la carpeta en Google Drive donde están las imágenes
input_folder = '/content/drive/My Drive/IMAGENESCOVID'
output_folder = '/content/drive/My Drive/processed_images'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Procesar cada imagen en la carpeta de entrada
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)

    # Leer la imagen
    image = cv2.imread(image_path)

    if image is None:
        print(f"No se pudo leer la imagen: {image_name}")
        continue

    # Aplicar Blur Filter
    blur = cv2.GaussianBlur(image, (15, 15), 0)

    # Aplicar Sobel Edge Detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    sobel = cv2.convertScaleAbs(sobel)  # Convertir para visualizar

    # Aplicar Canny Edge Detection
    canny = cv2.Canny(gray, 100, 200)

    # Aplicar Laplacian Edge Detection
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)  # Detectar bordes
    laplacian = cv2.convertScaleAbs(laplacian)   # Convertir para visualizar

    # Guardar los resultados en el sistema de archivos
    base_name = os.path.splitext(image_name)[0]
    cv2.imwrite(os.path.join(output_folder, f"{base_name}_blur.jpg"), blur)
    cv2.imwrite(os.path.join(output_folder, f"{base_name}_sobel.jpg"), sobel)
    cv2.imwrite(os.path.join(output_folder, f"{base_name}_canny.jpg"), canny)
    cv2.imwrite(os.path.join(output_folder, f"{base_name}_laplacian.jpg"), laplacian)

    print(f"Procesamiento completo para: {image_name}")

print(f"Todas las imágenes procesadas se han guardado en: {output_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Procesamiento completo para: 031.jpeg
Procesamiento completo para: COVID-00036.jpg
Procesamiento completo para: 023.jpeg
Todas las imágenes procesadas se han guardado en: /content/drive/My Drive/processed_images


In [27]:
import cv2
import os
import numpy as np
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Definir las rutas de la carpeta de entrada (donde están las imágenes) y la de salida (donde se guardarán las imágenes aumentadas)
input_folder = '/content/drive/My Drive/processed_images'  # Ruta de la carpeta con las imágenes en Google Drive
output_folder = '/content/drive/My Drive/augmented_images'  # Ruta de la carpeta donde se guardarán las imágenes aumentadas

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Funciones de aumento de datos
def augment_image(image):
    augmented = []

    # Rotaciones: 90°, 180°, 270°
    rows, cols = image.shape[:2]
    for angle in [90, 180, 270]:
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated = cv2.warpAffine(image, M, (cols, rows))
        augmented.append(rotated)

    # Escalado: 0.5x y 1.5x
    for scale in [0.5, 1.5]:
        scaled = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        augmented.append(scaled)

    # Traslaciones: izquierda, derecha, arriba, abajo
    for dx, dy in [(-50, 0), (50, 0), (0, -50), (0, 50)]:
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        translated = cv2.warpAffine(image, M, (cols, rows))
        augmented.append(translated)

    # Volteos: horizontal, vertical
    augmented.append(cv2.flip(image, 1))  # Volteo horizontal
    augmented.append(cv2.flip(image, 0))  # Volteo vertical

    # Filtro adicional: Escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
    augmented.append(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))  # Convertir a formato 3 canales para guardar

    return augmented

# Obtener la lista de archivos en la carpeta de entrada
image_files = os.listdir(input_folder)

# Procesar imágenes y aplicar aumento de datos
initial_count = len(image_files)
final_count = 0

for image_name in image_files:
    try:
        # Leer la imagen desde Google Drive
        image_path = os.path.join(input_folder, image_name)
        image = cv2.imread(image_path)

        if image is None:
            print(f"No se pudo leer la imagen: {image_name}")
            continue

        # Aplicar las transformaciones
        augmented_images = augment_image(image)
        final_count += len(augmented_images)

        # Guardar las imágenes aumentadas
        base_name = os.path.splitext(image_name)[0]
        for i, aug_image in enumerate(augmented_images):
            output_path = os.path.join(output_folder, f"{base_name}_aug_{i+1}.jpg")
            cv2.imwrite(output_path, aug_image)

        print(f"Procesamiento completo para: {image_name}, {len(augmented_images)} imágenes generadas.")
    except Exception as e:
        print(f"Error procesando {image_name}: {e}")

# Verificar si la carpeta tiene imágenes
if len(os.listdir(output_folder)) == 0:
    print("La carpeta de imágenes aumentadas está vacía. Verifica el proceso de escritura.")
else:
    print(f"Imágenes iniciales: {initial_count}")
    print(f"Imágenes después del aumento de datos: {initial_count + final_count}")
    print(f"Imágenes aumentadas guardadas en: {output_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Procesamiento completo para: 044_blur.jpg, 12 imágenes generadas.
Procesamiento completo para: 044_sobel.jpg, 12 imágenes generadas.
Procesamiento completo para: 044_canny.jpg, 12 imágenes generadas.
Procesamiento completo para: 044_laplacian.jpg, 12 imágenes generadas.
Procesamiento completo para: 074_blur.jpg, 12 imágenes generadas.
Procesamiento completo para: 074_sobel.jpg, 12 imágenes generadas.
Procesamiento completo para: 074_canny.jpg, 12 imágenes generadas.
Procesamiento completo para: 074_laplacian.jpg, 12 imágenes generadas.
Procesamiento completo para: 071_blur.jpg, 12 imágenes generadas.
Procesamiento completo para: 071_sobel.jpg, 12 imágenes generadas.
Procesamiento completo para: 071_canny.jpg, 12 imágenes generadas.
Procesamiento completo para: 071_laplacian.jpg, 12 imágenes generadas.
Procesamiento completo para: 031_blur.jpg, 12 imágenes gen